In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size = 64
data_dir = './data'
valid_size = 0.2

In [4]:
normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465],
    std=[0.2023, 0.1994, 0.2010],
)

valid_transform = transforms.Compose([
  transforms.Resize((227,227)),
  transforms.ToTensor(),
  normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    normalize,
])

train_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=train_transform,
)

valid_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=valid_transform,
)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, sampler=train_sampler
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, sampler=valid_sampler
)

100%|██████████| 170M/170M [00:06<00:00, 25.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
normalize = transforms.Normalize(
  mean=[0.485, 0.456, 0.406],
  std=[0.229, 0.224, 0.225],
)

transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    normalize,
])

test_dataset = datasets.CIFAR10(
    root=data_dir, train=False,
    download=True, transform=transform,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

Files already downloaded and verified


In [6]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, 11, 4, padding=0)
        self.conv2 = nn.Conv2d(96, 256, 5, 1, padding=2)
        self.conv3 = nn.Conv2d(256, 384, 3, 1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, 3, 1, padding=1)
        self.conv5 = nn.Conv2d(384, 256, 3, 1, padding=1)
        self.pool = nn.MaxPool2d(3, 2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.batchnorm = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(9216, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.batchnorm(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        out = self.fc3(x)
        return out

In [7]:
num_classes = 10
num_epochs = 20
learning_rate = 0.005

model = AlexNet(num_classes).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=4096, out_features=10, bias=True)
)


In [8]:
total_step = len(train_loader)

for epoch in range(num_epochs):
  model.train()
  for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in valid_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          del images, labels, outputs

      print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/20], Step [625/625], Loss: 1.5481
Accuracy of the network on the 5000 validation images: 47.04 %
Epoch [2/20], Step [625/625], Loss: 1.3623
Accuracy of the network on the 5000 validation images: 58.76 %
Epoch [3/20], Step [625/625], Loss: 0.7689
Accuracy of the network on the 5000 validation images: 60.82 %
Epoch [4/20], Step [625/625], Loss: 1.0508
Accuracy of the network on the 5000 validation images: 63.48 %
Epoch [5/20], Step [625/625], Loss: 0.8039
Accuracy of the network on the 5000 validation images: 68.02 %
Epoch [6/20], Step [625/625], Loss: 0.8924
Accuracy of the network on the 5000 validation images: 65.9 %
Epoch [7/20], Step [625/625], Loss: 0.9834
Accuracy of the network on the 5000 validation images: 69.15 %
Epoch [8/20], Step [625/625], Loss: 0.8536
Accuracy of the network on the 5000 validation images: 72.64 %
Epoch [9/20], Step [625/625], Loss: 0.8948
Accuracy of the network on the 5000 validation images: 73.68 %
Epoch [10/20], Step [625/625], Loss: 0.9050
Acc

In [9]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    del images, labels, outputs

  print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 76.09 %
